# Get Coast to Coast Assists and Buckets using PBP data


In [1]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "./figs/"

league = "nba"
season_yr = "2022"
season_type = "Regular Season"
games_id = pbp_season(league="nba",season_yr="2023",season_type="Regular Season")
games_list = pbp_games(games_id)

Number of games:  90


100%|██████████| 90/90 [00:02<00:00, 33.02it/s]

Number of bad games:  1


In [ ]:
year = int(season_yr)
player_dict = get_players(league = 'NBA', from_year = year, to_year = year)
team_dict = teams.get_teams() # Creating Team Dictionary
team_id = [x['id'] for x in team_dict ]

## Coast to Coast Assists

In [ ]:
from pbpstats.resources.enhanced_pbp import FieldGoal
from pbpstats.resources.enhanced_pbp import Rebound
pos_store = []
Rebound_pID = []
Assist_pID = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)) and possession_event.seconds_since_previous_event <= 5:
                if possession_event.previous_event.player1_id == possession_event.player2_id:
                    if not possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        print ("Rebound Player: {0}".format(possession_event.previous_event.player1_id))
                        Assist_pID.append(possession_event.player2_id)
                        print ("Assist Player: {0}".format(possession_event.player2_id))
                        ii += 1
                        # print("ii = {0}".format(ii))
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

In [ ]:
Player_Name = []
a = Assist_pID
Assist_pID1, Assists = np.unique(a, return_counts=True)  
for pID in Assist_pID1:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
Player_Name = list(itertools.chain(*Player_Name))

In [ ]:
data = pd.DataFrame({'Assist_pID':Assist_pID1,'Player_Name':Player_Name,'Assists':Assists})
data = data.sort_values(by=['Assists'],ascending=False)
data = data.reset_index(drop=True)
data.head()

## Coast to Coast Buckets

In [ ]:
pos_store = []
Rebound_pID = []
FGM_pID = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and not possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)) and possession_event.seconds_since_previous_event <= 8:
                if possession_event.previous_event.player1_id == possession_event.player1_id:
                    if not possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        print ("Rebound Player: {0}".format(possession_event.previous_event.player1_id))
                        FGM_pID.append(possession_event.player1_id)
                        print ("FGM Player: {0}".format(possession_event.player1_id))
                        ii += 1
                        print("ii = {0}".format(ii))
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

In [ ]:
Player_Name = []
a = FGM_pID
FGM_pID1, FGM = np.unique(a, return_counts=True)  
for pID in FGM_pID1:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
Player_Name = list(itertools.chain(*Player_Name))

In [ ]:
data = pd.DataFrame({'FGM_pID':FGM_pID1,'Player_Name':Player_Name,'FGM':FGM})
data = data.sort_values(by=['FGM'],ascending=False)
data = data.reset_index(drop=True)
data.head()